<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 603.8 MB 8.9 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [2]:
!git clone https://github.com/raymondwcs/learning_spacy/

fatal: destination path 'learning_spacy' already exists and is not an empty directory.


In [3]:
import spacy

sentences = [
  '大巴人以為跌一日博明天升',  #pos
  '入貨喇喂',  #pos
  '盡快掃貨',  #pos
  '樓主有料……聽日港股會升',  #pos
  '預咗侵侵佢輸硬,拜登上場,中美貿易戰結束,美跌港升',   #pos
  '呢幾日狂升',  #pos
  '大調整，股災危險', #neg
  '開始睇淡走貨啦', #neg
  '因為要殺熊', #neg
  '無人會聽你講 直至欲哭無淚為止', #neg
  '可以死守嗎', #neg
  '唔投降就等仆街了', #neg
]

In [4]:
nlp = spacy.load('zh_core_web_lg')
nlp.tokenizer.initialize(pkuseg_model="./learning_spacy/spacy_pkuseg/models")
nlp.tokenizer.pkuseg_update_user_dict(["大巴人", "博", "投降", "升"])

def myTokenizer(sentence):
  doc = nlp(sentence)
  list = []
  for token in doc:
    if token.pos_ == 'NOUN' or token.pos_ == 'VERB' or token.pos_ == 'PROPN':
      list.append(token.text)
  return(list)

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF
tfidf = TfidfVectorizer(tokenizer=myTokenizer)
weight = tfidf.fit_transform(sentences).toarray()
word = tfidf.get_feature_names()

# for i in range(len(weight)):
#   print('\nsentence {}'.format(i))
#   for j in range(len(word)):
#     print(word[j], weight[i][j])

df = pd.DataFrame(weight,columns=word)
print('Features: {}'.format(df.columns))
print('Number of features: {}'.format(len(df.columns)))
df

Features: Index(['一日', '上場', '中美', '人', '仆街', '侵侵', '入貨', '升', '博', '危險', '可以', '哭',
       '喇喂', '大巴人', '投降', '拜登', '掃貨', '料', '日', '明天', '有', '樓主', '欲', '死守',
       '殺熊', '港股', '無', '無淚為止', '直至', '睇淡', '等', '結束', '美跌港', '聽', '聽日', '股災',
       '要', '調整', '貿易戰', '走貨', '跌', '輸硬', '預咗'],
      dtype='object')
Number of features: 43


,一日,上場,中美,人,仆街,侵侵,入貨,升,博,危險,可以,哭,喇喂,大巴人,投降,拜登,掃貨,料,日,明天,有,樓主,欲,死守,殺熊,港股,無,無淚為止,直至,睇淡,等,結束,美跌港,聽,聽日,股災,要,調整,貿易戰,走貨,跌,輸硬,預咗
0,0.427817,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.291315,0.427817,0.00000,0.000000,0.000000,0.000000,0.427817,0.00000,0.000000,0.0,0.000000,0.000000,0.427817,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.427817,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.707107,0.000000,0.000000,0.00000,0.000000,0.000000,0.707107,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.291315,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.427817,0.000000,0.000000,0.427817,0.427817,0.000000,0.000000,0.000000,0.427817,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.427817,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.325065,0.325065,0.000000,0.00000,0.325065,0.000000,0.221348,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.325065,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.325065,0.325065,0.000000,0.000000,0.00000,0.000000,0.00000,0.325065,0.000000,0.000000,0.325065,0.325065
5,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.562839,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.826567,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000,0.57735,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.707107,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.707107,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.377964,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.377964,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.377964,0.000000,0.000000,0.000000,0.377964,0.377964,0.377964,0.000000,0.00000,0.000000,0.000000,0.377964,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [6]:
top_N = 3
for i in range(len(weight)):  # weights of each sentence
  sorted = np.argsort(weight[i])[::-1]  # reverse
  sorted = sorted[0:top_N]
  top_features = [word[w] for w in sorted if weight[i][w]]
  print('Sentence {}: {}'.format(i,top_features))

Sentence 0: ['一日', '博', '大巴人']
Sentence 1: ['入貨', '喇喂']
Sentence 2: ['掃貨']
Sentence 3: ['樓主', '聽日', '料']
Sentence 4: ['預咗', '侵侵', '輸硬']
Sentence 5: ['日', '升']
Sentence 6: ['危險', '調整', '股災']
Sentence 7: ['走貨', '睇淡']
Sentence 8: ['殺熊', '要']
Sentence 9: ['哭', '人', '欲']
Sentence 10: ['可以', '死守']
Sentence 11: ['等', '仆街', '投降']
